Run this notebook once you have trained the NER model (see `train.ipynb`)

# Load data from many markets

In [ ]:
import utils.scrapping
from sys import stderr
import os

In [ ]:
RAW_DATA_DIR = os.path.join("data", "raw")

if not os.path.exists(RAW_DATA_DIR):
    os.mkdir(RAW_DATA_DIR)

In [ ]:
markets = utils.scrapping.list_markets_in_city("São José dos Campos", "SP")

In [ ]:
len(markets)

In [ ]:
# save data for 5 markets
N = 5
for market in markets[:N]:
    print(f"retrieving products of {market.name}...")
    utils.scrapping.load_products_from_website(market, True)
    products = utils.scrapping.load_products_from_local_file(market)
    if products is None:
        print("FAILED", file=stderr)
        continue
    fname = os.path.join(RAW_DATA_DIR, market.slug_name + ".json")
    print(f"saving products of {market.name} into {fname}...")
    utils.scrapping.generate_data_file(products, fname)
print("DONE")

We can find the raw (i.e. not yet tagged) data in `data/raw/`

# Tag data

In [ ]:
from utils.ner import NERModel
import json
from tqdm import tqdm

In [ ]:
TAGGED_DATA_DIR = os.path.join("data", "tagged")

if not os.path.exists(TAGGED_DATA_DIR):
    os.mkdir(TAGGED_DATA_DIR)

In [ ]:
ner = NERModel("ner_v0")

In [ ]:
def tag_market(m):
    ner_resp = ner.retrieve_tags(m['product'])
    return {
        'product': m['product'],
        'tags': ner_resp['tags']
    }

for fname in os.listdir(RAW_DATA_DIR):

    print(f"tagging {fname}...")

    # read raw data:
    with open(os.path.join(RAW_DATA_DIR, fname), 'r') as f:
        market_products = eval(f.read()) # TODO dump as json, load as json; eval unsafe
    
    # tag data:
    market_products = list(tqdm(map(tag_market, market_products), total=len(market_products)))
    
    # save tagged data:
    with open(os.path.join(TAGGED_DATA_DIR, fname), 'w+') as f:
        json.dump(market_products, f, ensure_ascii=False)